In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv("songdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

#recommendation based on popularity, content base , collaborative filtering,hybrid 

#Clean

In [5]:
df['text'][0]

"Fuck nigga  \nMy blunt bigger than these lil niggas  \nKeep stuntin with your bitch ass  \nYou'll get jacked and Jilled, nigga  \nFuck wrong with these sissy niggas'  \nI'm smoking that Bill Bixby nigga  \nI'm spendin money, spendin money  \nBen Franklin dizzy nigga!  \nYour bitch on my dick, I told her I was busy  \nShe say 'Wayne on me, Wayne on me' cause yo ass need drizzlin  \nThat pussy came gift-wrapped, bow and a ribbon  \nAnd if she bounce that ass then I'm dribblin  \nThat pussy so wet, it turned into a Gremlin  \nThere's only one me, ain't no equivalent  \nShe give me brain, brain like trivia  \nI got a black bitch and a red bitch  \nI call them hoes Aunt Vivian  \nChopper knock your face off  \nBlack shades, Ray Charles  \nI be killing young hoes  \nI got your ho up in my graveyard  \nI ain't working with a full deck  \nBut I pull out that Ace card  \nShe grab that dick with 2 hands  \nLike she about to pray for it  \nClips hanging, no curfew  \nFuck you and who birthed you

In [6]:
df['text'].str.lower().replace(r'[^\w\s]',"").replace(r'\n','',regex=True)

0       fuck nigga  my blunt bigger than these lil nig...
1       again we stalk the stage  at mach speed we eng...
2       don't you let it all pass you by  looking thro...
3       mama's got a squeeze box  she wears on her che...
4       ft. taj jackson    baby don't speak  let your ...
                              ...                        
4995    verse 1  many are they increased that trouble ...
4996    missed the saturday dance  heard they crowded ...
4997    but all i want to know is a party year  let's ...
4998    vagabonds don't fall in love  they just wheel ...
4999    love makes me treat you the way that i do  bab...
Name: text, Length: 5000, dtype: object

In [7]:
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [ps.stem(w) for w in tokens] 
    return " ".join(stemming)


In [8]:
tokenization("loving love loved")

'love love love'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [10]:
tfid = TfidfVectorizer(stop_words='english')
matrix = tfid.fit_transform(df['text'])

In [11]:
matrix.shape

(5000, 23870)

In [12]:
similarity = cosine_similarity(matrix)

In [13]:
similarity[0]

array([1.        , 0.02615769, 0.02890633, ..., 0.02239563, 0.02833039,
       0.04946246])

In [16]:
df['song'][0]

'Cashed Out'

In [17]:
df['song']=='Cashed Out'

0        True
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: song, Length: 5000, dtype: bool

In [18]:
df[df['song']=='Cashed Out']

,artist,song,text
0,Lil Wayne,Cashed Out,Fuck nigga \nMy blunt bigger than these lil n...


In [19]:
list(enumerate(similarity[0]),reverse=False,key=lambda x:x[1])

[(0, 0.9999999999999999),
 (1, 0.0261576945268078),
 (2, 0.0289063260800727),
 (3, 0.028304678428863896),
 (4, 0.04315366722265628),
 (5, 0.032841665268607295),
 (6, 0.015808701635738793),
 (7, 0.00862566242882986),
 (8, 0.06490205799180251),
 (9, 0.02600480204279241),
 (10, 0.019039293384291545),
 (11, 0.005713488893444924),
 (12, 0.016622958457964784),
 (13, 0.0012556659397750204),
 (14, 0.017999938923646437),
 (15, 0.006353904302054428),
 (16, 0.033329630554185204),
 (17, 0.04699147672016146),
 (18, 0.0024028137606306576),
 (19, 0.014302606587042844),
 (20, 0.010062523776127489),
 (21, 0.018545452789904153),
 (22, 0.027373316817844596),
 (23, 0.001733563186311227),
 (24, 0.010000947458192458),
 (25, 0.022725677169503796),
 (26, 0.028289210577054368),
 (27, 0.0477866180869493),
 (28, 0.03174181977784651),
 (29, 0.0032607037509412907),
 (30, 0.009956694403754397),
 (31, 0.03540193013027229),
 (32, 0.018496586520549363),
 (33, 0.04349008565664324),
 (34, 0.0017014832071519355),
 (35, 0

In [27]:
def recommendation(song):
    idx = df[df['song']==song].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=False,key=lambda x:x[1])
    songs = []
    for i in distances[1:21]:
        songs.append(df.iloc[i[0]].song)
    return songs
    

In [29]:
recommendation('Winter Time')

['The Killing Road',
 'Squeeze Box',
 'Heaven',
 'Life Itself',
 'Maria',
 'Carry On',
 'I Want',
 'Oh, Pretty Woman',
 'Pepsi World Cup',
 'Head Injury',
 'The Big Picture',
 "You're Everything",
 "Don't Judge Me",
 'I Wanna Little Love',
 'Since I Met You Baby',
 'Christmas Blues',
 'End Of The End',
 'Everybody I Love You',
 'Falling Down',
 'Horizontal']

In [37]:
import pickle
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))